In [1]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4],
  autocommit=True
)

mycursor = mydb.cursor(dictionary=True)

In [8]:
def exec_q(qT):
  mycursor.execute(qT)
  myresult = mycursor.fetchall()
  jD = []
  for x in myresult:
    jD.append(x['json_result'])
  JReturn = '['+','.join(jD)+']'
  return JReturn

def exec_to_result(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def get_erm_result(code,v='',limit=''):
    verQ = ' TRUE ' if v=='' else ' versi = "'+v+'" '
    limQ = '' if limit=='' else ' LIMIT '+limit 
    return exec_q("select json_result from primaprod.mr_result inner join primaprod.mr_form on \
id_form=mr_form.id where kode like '"+code+"' AND "+verQ+limQ)

def get_erm_result_from_RM(rm):
  #dapatkan barisan DR
  rawPKP = exec_to_result("select poli_kunjungan_pasien.attr_tambahan from poli_kunjungan_pasien inner join pendaftaran\
    using (no_reg) where pendaftaran.no_rm = '"+rm+"' and poli_kunjungan_pasien.status_ok = 1\
      and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_erm_result_from_NOREG(nr):
  #dapatkan barisan attr
  rawPKP = exec_to_result("select attr_tambahan from poli_kunjungan_pasien where no_reg = '"+nr+"' and status_ok = 1\
     and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_erm_result_from_DR(dr):
  rawPKP = exec_to_result("select attr_tambahan from poli_kunjungan_pasien where id = '"+dr+"' and status_ok = 1\
    and attr_tambahan like '%\"emr\":%'")
  return get_attr_emr(rawPKP)

def get_attr_emr(attrs):
  EMRid = []
  for i in attrs:
    attr = json.loads(i['attr_tambahan'])
    for j in attr['emr']:
      EMRid.append(j)
  mycursor.execute('select mr_result.id,json_result,kode,user_created from primaprod.mr_result inner join primaprod.mr_form on \
id_form=mr_form.id where mr_result.id in ("'+'","'.join([str(int) for int in EMRid])+'")')
  return parse_code_to_key(mycursor.fetchall()) 

def parse_code_to_key(dic):
  rDic = {}
  for i in dic:
    jR = json.loads(i['json_result'])
    for k in jR:
      rDic[i['kode']+'.'+str(i['user_created'])+'.'+str(i['id'])+'.'+k] = []
      rDic[i['kode']+'.'+str(i['user_created'])+'.'+str(i['id'])+'.'+k].append(jR[k])
  return rDic


In [13]:
DFresult = pd.read_json(get_erm_result('RM-B01','','1'))

In [14]:
DFresult

,alergi_lainnya,ats_10,ats_22,ats_31,ats_32,ats_42,ats_43,ats_53,gcs,jenis_kelamin,nadi,nafas_,nama,riwayat_alergi_makanan,riwayat_alergi_obat,saturasi_oksigen,suhu_,tekanan_darah
0,,1 | Henti Jantung && 2 | Henti Nafas && 3 | RR...,2 | Hemiparese akut dan penurunan kesadaran,1 | Batuk berdahak disertai demam dan sesak,2 | Perdarahan sedang,2 | Aspirasi benda asing tanpa gangguan pernaf...,2 | Mual/diare tidak Dehidrasi,2 | Riwayat penyakit risiko rendah sekarang ti...,,L,34,34,AGUS W,,,,34,120/90


In [ ]:
get_erm_result_from_RM('2586113')

In [16]:
dtdic = get_erm_result_from_DR('3296830')
df = pd.DataFrame(dtdic)

In [17]:
dtdic

{'RM-B03.28103263.1407613.ruangan': ['TULIP LT.1 - NON INFEKSI'],
 'RM-B03.28103263.1407613.informasi_di_dapat_dari': ['AUTO-ANAMNESIS'],
 'RM-B03.28103263.1407613.pekerjaan_': ['KARYAWAN SWASTA'],
 'RM-B03.28103263.1407613.cara_bayar': ['JKN'],
 'RM-B03.28103263.1407613.tgl_jam': ['07-05-2022 20:12'],
 'RM-B03.28103263.1407613.tanggal_jam_pemeriksaan': ['07-05-2022 22:09'],
 'RM-B03.28103263.1407613.fasilitas_kesehatan_tk1': ['kel cipinang'],
 'RM-B03.28103263.1407613.praktek_mandiri_': [''],
 'RM-B03.28103263.1407613.status_alergi': ['TIDAK'],
 'RM-B03.28103263.1407613.ya_sebutkan': [''],
 'RM-B03.28103263.1407613.d_d_keluhan_utama_lainnya': [''],
 'RM-B03.28103263.1407613.d_d_keluhan_tambahan': [''],
 'RM-B03.28103263.1407613.d_d_riwayat_perjalanan_penyakit_saat_ini': [''],
 'RM-B03.28103263.1407613.alasan_dirawat_': ['DM'],
 'RM-B03.28103263.1407613.riwayat_kecelakaan_': ['Tidak ada'],
 'RM-B03.28103263.1407613.riwayat_di_operasi': ['tidak ada`'],
 'RM-B03.28103263.1407613.d_d_riwa